# 0) installation de package manquant
1. appel système avec le point d'exclamation
2. appel de l'installateur de package : 

-pip (python classique )

-conda (python anaconda)

3. paramétrage de l'installateur avec l'option pour accepter les conditions par défaut d'installation des packages :  -y

Exemple : !conda install -y regex

#### Sinon, aller voir dans anaconda navigator pour une interface graphique

In [1]:
!conda install -y regex

Solving environment: ...working... done

# All requested packages already installed.



## usage de jupyter : fusion de cellules 
# MAJ + M

# 1) regular expressions

In [2]:
import regex 
url ="https://github.com/romainjouin/esilv_5a.git"

## 1.1) regex greedy : le .* prend tout ce qu'il peut

In [3]:
pattern = "https://.*\.git"
model   = regex.compile(pattern)
text    = url + " coucou .gita"+url
model.findall(text)

['https://github.com/romainjouin/esilv_5a.git coucou .gitahttps://github.com/romainjouin/esilv_5a.git']

## 1.2) regex lazzy : le .*? s'arrête dès qu'il peut

In [4]:
recherche = ".*?"
pattern   = f"https://{recherche}\.git"
model     = regex.compile(pattern)
model.findall(text)

['https://github.com/romainjouin/esilv_5a.git',
 'https://github.com/romainjouin/esilv_5a.git']

# 2) webscrapping

In [5]:
import requests as re         # module de requête http
page = re.request('get', url) # on récupère une page 
html = page.text              # on récupère le contenu html de la page

### type de html => string => on peut l'analyser avec des regex !

In [6]:
type(html)

str

## 2.1) regex pour la récupération des liens 
un lien html => (<a href=> < /a>)

In [7]:
import regex

In [8]:
pattern_lien = "<a.*/a>"
model        = regex.compile(pattern_lien)
liens        = model.findall(html)
len(liens)#[:3]

145

Question d'axel: comment récupérer les liens qui ne comprennent pas le mot github ?
- changer la regex en ajoutant un point d'exclamation (signifie la négation) avant le .* (idée de nicolas / alexis pense que ça march'ra pas mais il ne veut pas dire de bêtises)
- Simon : prendre tous les liens, une seconde regex avec les .git => filtre inversé avec la seconde regex
- Lindsay : ne pas utiliser une seconde rgex est il envisageable ?? Très bonne idée !! -> comment faire ? Romain propose de faire un filtre

## 2.2) regex sur regex (simon)

In [9]:
seconde_regex = ".*?\.git"

second_model = regex.compile(seconde_regex)
liens_git = second_model.findall(html)
liens_git[:3]

['  <link rel="dns-prefetch" href="https://github.git',
 '  <link rel="dns-prefetch" href="https://avatars0.git',
 '  <link rel="dns-prefetch" href="https://avatars1.git']

In [10]:
[lien for lien in liens if not second_model.findall(lien)][:3]

['<a href="#start-of-content" class="px-2 py-4 bg-blue text-white show-on-focus js-skip-to-content">Skip to content</a>',
 '<a href="/features" class="py-2 lh-condensed-ultra d-block link-gray-dark no-underline h5 Bump-link--hover" data-ga-click="(Logged out) Header, go to Features">Features <span class="Bump-link-symbol float-right text-normal text-gray-light">&rarr;</span></a>',
 '<a href="/features/code-review/" class="py-2 lh-condensed-ultra d-block link-gray no-underline f5" data-ga-click="(Logged out) Header, go to Code review">Code review</a>']

## 2.3) filtre (romain)

In [11]:
l1 = list(filter(lambda lien: ".git" not in lien, liens)) 
l2 = [lien for lien in liens if not second_model.findall(lien)]

In [12]:
len(l1), len(l2)

(130, 130)

In [13]:
l1==l2

True

# 3) mieux que les regex : la soupe !

In [14]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(html)

## 3.1) trouver tous les liens

In [16]:
selector = "a"
liens    = soup.find_all(selector)
len(liens)

189

## 3.3) trouver les éléments du dom qui ont une certaine classe

In [17]:
selector = "px-2"
nodes    = soup.find_all(class_=selector)
len(nodes)

4

## 3.4) trouver les éléments avec deux classes données

In [18]:
selector = {'class':['py-2', 'lh-condensed-ultra']}
nodes    = soup.find_all(True, selector)
len(nodes)

120

# 4) exploration des noeuds

In [19]:
un_noeud = nodes[0]

In [20]:
type(un_noeud)

bs4.element.Tag

## 4.1) les noeuds ont des textes

In [21]:
un_noeud.text

'\n\n\n\n\n\n\n\n\n\n                Sign\xa0up\n              \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    Why GitHub?\n                    \n\n\n\n\nFeatures →\n\nCode review\nProject management\nIntegrations\nActions\nPackages\nSecurity\nTeam management\nHosting\nMobile\n\n\nCustomer stories →\nSecurity →\n\n\n\n\n\nTeam\n\n\nEnterprise\n\n\n\n\n                    Explore\n                    \n\n\n\n\n\nExplore GitHub →\n\nLearn & contribute\n\nTopics\nCollections\nTrending\nLearning Lab\nOpen source guides\n\nConnect with others\n\nEvents\nCommunity forum\nGitHub Education\nGitHub Stars program\n\n\n\n\n\nMarketplace\n\n\n\n\n                    Pricing\n                    \n\n\n\n\nPlans →\n\nCompare plans\nContact Sales\n\n\nNonprofit →\nEducation →\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        In this repository\n      \n\n        All GitHub\n      \n↵\n\n\n      Jump to\n      ↵\n\n\n\n\n\n\nNo suggested jump to results\n\n\n\n\n\n\n\n\n\n\n\n

In [22]:
len(un_noeud.text)

1099

## 4.2) on peut se balader dans le DOM 

In [23]:
len(un_noeud.parent.text)

1120

In [24]:
for child in un_noeud.children:
    try:
        print(child)
    except Exception as e:
        pass



<div class="container-xl d-lg-flex flex-items-center p-responsive">
<div class="d-flex flex-justify-between flex-items-center">
<a aria-label="Homepage" class="mr-4" data-ga-click="(Logged out) Header, go to homepage, icon:logo-wordmark" href="https://github.com/">
<svg aria-hidden="true" class="octicon octicon-mark-github text-white" height="32" version="1.1" viewbox="0 0 16 16" width="32"><path d="M8 0C3.58 0 0 3.58 0 8c0 3.54 2.29 6.53 5.47 7.59.4.07.55-.17.55-.38 0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94-.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53.63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66.07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95 0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12 0 0 .67-.21 2.2.82.64-.18 1.32-.27 2-.27.68 0 1.36.09 2 .27 1.53-1.04 2.2-.82 2.2-.82.44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15 0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48 0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55.38A8.013 8.013 0 0016 8c0-4.42-3.58-8-8-8z" fill-rule="evenodd"></pat

In [25]:
_ = list(un_noeud.children)[1]
type(_)

bs4.element.Tag

In [26]:
len(list(_.children))

5

# 5) analyse météo
url sur https://github.com/romainjouin/esilv_5a/blob/master/url_meteo

In [27]:
url   = "https://www.infoclimat.fr/observations-meteo/archives/7/septembre/2019/paris-montsouris/07156.html"
page  = requests.request("get", url )
html  = page.text
soup  = BeautifulSoup(html)

NameError: name 'requests' is not defined

In [ ]:
html

## 5.1) vérifier la tête de la page récupérée en l'enregistrant dans un fichier afin de pouvoir l'ouvrir via un navigateur

In [ ]:
with open("meteo.html", "a") as f:
    f.writelines(html)

In [ ]:
soup

# 5.2) récupérer le tableau de données avec les CSS Selectors
Les sélecteurs CSS permettent d'extraire une partie d'une page web, ils sont officiellement décrits par le w3C ici :  
https://www.w3.org/TR/selectors-3/

Des résumés existent sur le web, notamment ces deux suivants :   
https://gist.github.com/magicznyleszek/809a69dd05e1d5f12d01  
https://www.dropbox.com/s/h2hni9o1m1di989/CSS%20selectors%20cheatsheet.pdf?dl=0

### 5.2.1) récupérer toutes les lignes

In [ ]:
css_selector = f'#resptable-releves > tbody > tr'
table_row    = soup.select(css_selector)
table_row

In [ ]:
len(table_row)

### 5.2.2) récupérer les data dans une ligne et une colonne données

In [ ]:

ligne_n         =  1
colonne_n       =  3
classe          =  "span.tipsy-trigger" 

classe_tableau  =  "#resptable-releves"
contenu_tableau =  "tbody" 
lignes          = f"tr:nth-child({ligne_n})"
colonnes        = f"td:nth-child({colonne_n})"

css = " > ".join([classe_tableau, contenu_tableau, lignes, colonnes, classe])
soup.select(css)[0].text

### 5.2.3) récupérer les data dans toutes les lignes et toutes les colonnes

In [ ]:
for ligne_n in range(25):
    print("*"*10)
    print(f"ligne : {ligne_n}")
    for colonne_n in range(1, 10):
        print("-"*10)
        try:
            print(f"ligne : {ligne_n} colonne :{colonne_n}" )
            classe_tableau  =  "#resptable-releves"
            contenu_tableau =  "tbody" 
            lignes          = f"tr:nth-child({ligne_n})"
            colonnes        = f"td:nth-child({colonne_n})"

            css = " > ".join(filter(lambda x: x, [classe_tableau, contenu_tableau, lignes, colonnes, classe]))
            print(css)
            print(soup.select(css)[0].text)
        except Exception as e:
            print(e)

### 5.2.4) Créons une fonction paramétrisable

In [ ]:
def valeur_dans_la_ligne_et_colonne(ligne_n, colonne_n, soup):
    classe_tableau  =  "#resptable-releves"
    contenu_tableau =  "tbody" 
    try:
        lignes          = f"tr:nth-child({ligne_n})"
        colonnes        = f"td:nth-child({colonne_n})"
        css = " > ".join(filter(lambda x: x, [classe_tableau, contenu_tableau, lignes, colonnes]))
        return soup.select(css)[0].text
    except Exception as e:
        return ""

In [ ]:
for ligne_n in range(1, 25):
    print("*"*10)
    print(f"ligne : {ligne_n}")
    for colonne_n in range(1, 11):
        print("-"*10)
        r = valeur_dans_la_ligne_et_colonne(ligne_n, colonne_n, soup)
        print(r)

# 6) Enregistrons le résultat dans une dataframe


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
for ligne_n in range(1, 25):
    print("*"*10)
    print(f"ligne : {ligne_n}")
    values = []
    for colonne_n in range(1, 11):
        print("-"*10)
        r = valeur_dans_la_ligne_et_colonne(ligne_n, colonne_n, soup)
        values.append(r)
        print(r)
    df[ligne_n] =values

In [ ]:
df

## Il est plus simple de transposer la matrice

In [ ]:
df = df.T

In [ ]:
df

## 6.1) Récupérons les entêtes de colonnes


In [ ]:
headers = soup.select("#resptable-releves > thead > tr > th")

In [ ]:
nom_des_colonnes = [header.get_text().strip() for header in headers]
nom_des_colonnes 

In [ ]:
df.columns = nom_des_colonnes 
df

## 6.2) récupérons les heures locales manquantes

In [ ]:
heures = [soup.select(f"#resptable-releves > tbody > tr:nth-child({n_ligne}) > th > span")[0].text 
          for n_ligne in range(1,25)]
heures

In [ ]:
df[df.columns[0]] = heures
df

## 6.3) transformons la colonne d'humidité qui est une colonne de string en une colonne de int

In [ ]:
import string

In [ ]:
col_percent = df["Humidité"]
col_percent = col_percent.apply(lambda x: int(x[:-1]))
df.loc[:,"Humidité"] = col_percent
df

## 6.4) faisons un graphique d'évolution de l'humidité sur la journée

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns

In [ ]:
df["Humidité"].plot(title="% d'humidité")
sns.despine()

# 7) EXAMPLE 2: Visualiser les donations faites au téléthon

## 7.1) récupérons la data

In [ ]:
import requests
url        = "https://www.afm-telethon.fr/telethon/bref/parrains-resultats-telethon-1379"
requesting = requests.get(url)
requesting.status_code
soup       = BeautifulSoup(requesting.content, "html.parser")

In [ ]:
year               = [td.get_text() for td in soup.select("tr td")[3::3]]
referrer           = [td.get_text() for td in soup.select("tr td")[4::3]]
amount_collected   = [td.get_text() for td in soup.select("tr td")[5::3]]

## 7.2) mettons la dans une datFrame

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'year'             : year,
    'referrer'         : referrer,
    'amount_collected' : amount_collected
})

In [ ]:
df.head(10)

## 7.3) nettoyons les types des colonnes

In [ ]:
df.dtypes

In [ ]:
df['year'] = df.year.astype(int) #on cast en integer
#on récupère que les chiffres dans le prix, on cast en int et on divise par 1m pour avoir le prix en millions
df['amount_collected'] = df.amount_collected.str.replace('\D', '').astype(int)/10000000
#on supprime les espaces avant et après les noms
df.referrer = df.referrer.str.strip()

In [ ]:
df.dtypes

## 7.4) visualisons

In [ ]:

%matplotlib inline

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# DISPLAYING FEATURES
fig, ax = plt.subplots(figsize=(15, 10), ncols=1, nrows=1) #fig containing a subplot 'ax' using seaborn package

#color theme
flatui = ["#7F7FFF","#CCCCFF"] # some great colors 
sns.set_style("whitegrid") # to have a style in the background grid

#the plot is a barplot, x for each year in absciss, y in ordinate
#we add the palette of colors and assign the the defined ax
ax = sns.barplot(x="year", y="amount_collected", 
                 data=df, palette=flatui, ax=ax)

#rotation of the x labels
ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=90)
#title and subtitle
ax.text(x=0.5, y=1.05, s='Prix investi dans le Telethon', fontsize=18, weight='bold', ha='center', va='bottom', transform=ax.transAxes)
ax.text(x=0.5, y=1.01, s='En France depuis 1987', fontsize=14, alpha=0.75, ha='center', va='bottom', transform=ax.transAxes)
#grid labels
ax.set_xlabel('Year',fontsize=16)
ax.set_ylabel('Amount Collected', fontsize=16)

#adding something fancy: the actual digit in donations 2/3 centered in each barplot
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height*2/3, '{}'.format(round(height,2)), 
            color='white', ha="center", rotation=90, weight='bold')